## 3.4 Assignment

the task is to implement a dfs for the alpha cliques

in dolphins.txt the first line is the number of nodes 62
the following lines are starts and end vertices for undirected edges

The other file is dolphininfo.txt where the i:th line tells the name and sex
(F=female, M=male, U=unspecified) of dolphin (node) number i.


$ deg_S(v)  $   is the degree of v in set S
$ deg_V(v)  $   is the degree of v in the entire graph


a set S is an $ \alpha-clique $   if the smallest degree node in S, divided by the number of nodes - 1, is $ \geq \alpha $

### Pruning

even though $ \alpha-cliequedness $ is not monotonic, it is still possible to prune all supersets according to the formula. 


In [186]:
import numpy as np


alpha = 0.6

In [187]:
def read_graph(file_path):
    adjacency_list = {}
    with open(file_path, 'r') as file:
        lines = file.readlines()
        # Extract the number of nodes from the first line
        first_line = lines[0].strip()
        num_nodes = int(first_line.split()[0])
        
        # Initialize adjacency list with empty lists for each node
        for node in range(1, num_nodes + 1):
            adjacency_list[node] = []
        
        # Parse each edge and populate the adjacency list
        for line in lines[1:]:
            parts = line.strip().split()
            if len(parts) == 2:
                u, v = int(parts[0]), int(parts[1])
                adjacency_list[u].append(v)
                adjacency_list[v].append(u)  # Because the graph is undirected
            else:
                raise Exception('wrong number')
    return adjacency_list


In [188]:
# Read the graph from the file
graph_toys = read_graph('toyset.txt')
print(len(graph_toys))

# Print the adjacency list
for node in graph_toys:
    print(f"{node}: {graph_toys[node]}")


5
1: [2, 4]
2: [1, 3, 4]
3: [2, 4, 5]
4: [1, 2, 3]
5: [3]


In [189]:
def is_alpha_clique(graph, set_of_nodes, alpha: float):
    mindegrees = {node: 0 for node in set_of_nodes}

    S = len(set_of_nodes)
    if S < 3:
        return False
    for node in set_of_nodes: 
        mindegrees[node] = len([neighbor for neighbor in graph[node] if neighbor in set_of_nodes])
    condition = min(mindegrees.values()) / (S - 1) >= alpha
    return condition



In [190]:

def is_limited_by_theorem(graph, set_of_nodes, alpha: float):
    mindegrees = {node: 0 for node in set_of_nodes}

    S = len(set_of_nodes)
    if S < 3:
        return False
    for node in set_of_nodes: 
        mindegrees[node] = len([neighbor for neighbor in graph[node] if neighbor in set_of_nodes])
    function_S = min(mindegrees.values()) / (S - 1)
    # the condition holds for the largest degree in the set S, considering all nodes around it


    all_degrees = {node: 0 for node in set_of_nodes}
    for node in set_of_nodes: 
        all_degrees[node] = len([neighbor for neighbor in graph[node]])
    max_deg = max(all_degrees.values())
    upper_bound = 1  - (max_deg * (1 - alpha)) / ((S - 1)  *alpha)
    condition = function_S < upper_bound
    return condition


In [191]:
import numpy as np
from typing import Dict, List, Set, Tuple



def dfs(at: int, 
        visited: np.ndarray, 
        graph: Dict[int, List[int]], alpha, global_setof_cliques, initial_call: bool = True, 
        debug: bool = False) -> None:
    """
    Performs a recursive Depth-First Search (DFS) starting from the given node.

    This function explores all possible paths in the graph by allowing backtracking.
    Each recursive call receives its own copy of the visited array to prevent
    interference between different exploration paths.

    Args:
        at (int): The current node being visited.
        visited (np.ndarray): A boolean array indicating which nodes have been visited.
        graph (Dict[int, List[int]]): The adjacency list representation of the graph.
        debug (bool): If True, prints debug information. Defaults to False.

    Returns:
        None
    """
    if debug and initial_call:
        print(f"ALGO Begins for Node {at}")

    # If the node is already visited in the current path, skip it

    # if debug:
    #     print(f"Visiting node {at}.")

    # Create a copy of the visited array for the current path
    visited = visited.copy()
    visited[at - 1] = True
    if debug:
        print(f"Node {at} marked as visited. Updated visited: {visited}")
    
    visited_nodes = tuple(sorted([i + 1 for i, value in enumerate(visited) if value]))
    if debug:
        print(f"\nTRYING OUT SET {visited_nodes}")
    
    if is_limited_by_theorem(graph=graph, set_of_nodes=visited_nodes, alpha=alpha):
        if debug:
            print("----------------------------------------")
            print(f" THEOREM STOP -- BACKTRACKING FROM {at}")
            print("----------------------------------------")
        return 
    if is_alpha_clique(graph=graph, set_of_nodes=visited_nodes, alpha=alpha): # Writing .add stops duplicate entries from entering
        global_setof_cliques.add(visited_nodes)

    # Retrieve neighbors; use get to handle nodes with no neighbors
    neighbours = graph.get(at, [])
    if debug:
        print(f"\nNode {at} has neighbors: {neighbours}")

    for neighbour in neighbours:
        neighbour_is_visited = visited[neighbour - 1]
        if debug:
            print(f"Node {at} -> Attempting to visit neighbor {neighbour}", end='')
            if neighbour_is_visited:
                print("==> already visited")
            else:
                print("==> accepted")


        if not neighbour_is_visited:
            dfs(neighbour, visited, graph, alpha=alpha, global_setof_cliques=global_setof_cliques, initial_call=False, debug = debug)

    if debug:
        print(f"All neighbors of node {at} have been processed. Backtracking from node {at}.\n")



In [192]:
# Initialize the visited array with all False (unvisited)
initial_visited: np.ndarray = np.zeros(len(graph_toys), dtype=bool)
global_setof_cliques: Set[Tuple[int, ...]] = set()


for node_index in graph_toys.keys():
    dfs(node_index, initial_visited, graph_toys, alpha=alpha, global_setof_cliques=global_setof_cliques, initial_call=True, debug=True)
global_setof_cliques

ALGO Begins for Node 1
Node 1 marked as visited. Updated visited: [ True False False False False]

TRYING OUT SET (1,)

Node 1 has neighbors: [2, 4]
Node 1 -> Attempting to visit neighbor 2==> accepted
Node 2 marked as visited. Updated visited: [ True  True False False False]

TRYING OUT SET (1, 2)

Node 2 has neighbors: [1, 3, 4]
Node 2 -> Attempting to visit neighbor 1==> already visited
Node 2 -> Attempting to visit neighbor 3==> accepted
Node 3 marked as visited. Updated visited: [ True  True  True False False]

TRYING OUT SET (1, 2, 3)

Node 3 has neighbors: [2, 4, 5]
Node 3 -> Attempting to visit neighbor 2==> already visited
Node 3 -> Attempting to visit neighbor 4==> accepted
Node 4 marked as visited. Updated visited: [ True  True  True  True False]

TRYING OUT SET (1, 2, 3, 4)

Node 4 has neighbors: [1, 2, 3]
Node 4 -> Attempting to visit neighbor 1==> already visited
Node 4 -> Attempting to visit neighbor 2==> already visited
Node 4 -> Attempting to visit neighbor 3==> alread

{(1, 2, 3, 4), (1, 2, 4), (2, 3, 4)}